In [11]:
import ee
import io
import requests
import numpy as np
from datetime import date, timedelta
import matplotlib.pyplot as plt

In [12]:
ee.Initialize()

In [13]:
MAX_RES_SCALE = 10 # 1 pixel = 10x10m

def fetch_DWData(region_shape: ee.Geometry.Polygon, start_t:str, end_t:str):
    search_filter = ee.Filter([ee.Filter.bounds(region_shape), ee.Filter.date('2021-04-02', '2021-04-03')]);
    dwCol = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filter(search_filter)

    latest_partition = dwCol.first().toInt8()

    data_url = latest_partition.getDownloadURL({
        'bands': ['label'],
        'region': region_shape,
        'scale': MAX_RES_SCALE,
        'format': 'NPY'
    })

    response = requests.get(data_url)

    if response.status_code == 200:
        raw_data = np.load(io.BytesIO(response.content))
        return raw_data
    
    return None


In [14]:
def clean_data(raw:np.array):
    new_data = []
    for row in raw:
        new_row = []
        for item in row:
            new_row.append(item[0])
        new_data.append(new_row)

    return np.array(new_data)

In [15]:
VALUE2BAND  = { 0 : 'water',
                1 : 'trees',
                2 : 'grass',
                3 : 'flooded_vegetation',
                4 : 'crops',
                5 : 'shrub_and_scrub',
                6 : 'built',
                7 : 'bare',
                8 : 'snow_and_ice' }

BAND2VALUE  = { 'water' : 0,
                'trees' : 1,
                'grass' : 2,
                'flooded_vegetation' : 3,
                'crops' : 4,
                'shrub_and_scrub' : 5,
                'built' : 6,
                'bare' : 7,
                'snow_and_ice' : 8 }

                
ELEGIBLE_BANDS = ['grass', 'crops', 'shrub_and_scrub', 'bare']

HECT_CONVERTION_FACTOR = .001

def isolate_band(sat_data : np.array, band : str): 
    data_filter = sat_data == BAND2VALUE[band]
    return sat_data[data_filter] / BAND2VALUE[band]

def band_area(sat_data : np.array, band : str):
    band_pixels = isolate_band(sat_data, band)
    band_pixels = band_pixels.flatten()

    return np.add.reduce(band_pixels)

In [16]:
def calc_approx(region_shape: ee.Geometry.Polygon):
    today = date.today()
    tolerance_past = today - timedelta(days=30)

    start_t_str = tolerance_past.strftime("%Y-%m-%d")
    end_t_str = tolerance_past.strftime("%Y-%m-%d")

    raw_array = fetch_DWData(region_shape, start_t_str, end_t_str)
    if raw_array is None: return None

    dwData = clean_data(raw_array)
    elegible_area = 0

    for band in ELEGIBLE_BANDS:
        halve = 0.5 if band == 'crops' else 1
        elegible_area += band_area(dwData, band)

    elegible_area *= HECT_CONVERTION_FACTOR

    carbon_calc = 13.815 * elegible_area
    car_calc = carbon_calc / 0.43

    return {'elegible_area' : elegible_area,
            'carbon_calc'   : carbon_calc,
            'car_calc'      : car_calc} 

In [17]:
test_polygon = ee.Geometry.Polygon(
    [
        [
            [-103.1913812014911,20.33034573665963],
            [-103.1922818923659,20.32761306717687],
            [-103.1880223393225,20.32661425447412],
            [-103.1883963909232,20.33111088627422],
            [-103.1913812014911,20.33034573665963]
        ]
    ],
    None,
    False
)

results = calc_approx(test_polygon)

In [27]:
response = requests.get('https://v6.exchangerate-api.com/v6/5e3481394f866fbf4bf07bac/latest/USD')
now_mxn_rate = (response.json())['conversion_rates']['MXN']

carbon_profit = results['carbon_calc']*now_mxn_rate*carbon_rate

In [28]:
carbon_profit

4136.448272625

In [25]:
results

{'elegible_area': 1.492,
 'carbon_calc': 20.61198,
 'car_calc': 47.93483720930232}